In [84]:
import numpy

In [165]:
def distance( x, c ):
    
    
    d = numpy.sqrt( sum( sum( ( x - c )**2 ) ) )
    
    return d

In [155]:
def kmean( x, k, max_iters ):
    
    centroid = x[ numpy.random.choice( range( len( x ) ), k, replace = False ) ]
    ites = 0
    diff = 1

    while ( diff != 0 ) and ( ites <= max_iters ):


        clusters = [ [] for i in range( len( centroid ) ) ]

        for i in range( len( x ) ):  # Go through each data point

            dist = []

            for j in range( len( centroid ) ):

                dist.append( distance( centroid[ j ], x[ i ] ) )

            clusters[ numpy.where( dist == min( dist ) )[ 0 ][ 0 ] ].append( x[ i ] ) 

        centroid_minus = centroid.copy()
        centroid = []

        for i in range( len( clusters ) ):

            centroid.append( numpy.mean( clusters[ i ], axis = 0 ) )

        if ( sum( centroid ) - sum( centroid_minus ) ).all() == 0:
            diff = 0
        ites += 1

    std = []
    for i in range( len( clusters ) ):
        std.append( numpy.std( clusters[ i ] ) )
    std = numpy.array( std )


    return centroid, std

In [286]:
class NeuralNet:
    
    def __init__( self, x, y, num_classes, k, max_iters ):
        
        self.input = x
        self.y = y
        self.num_classes = num_classes
        self.k = k
        self.max_iters = max_iters
    
    def one_hot_encode( self, y, num_classes ):
        
        one_hot_arr = numpy.zeros( ( len( y ), num_classes ) )
        
        for i in range( len( y ) ):
            
            one_hot_arr[ i ][ int( y[ i ] ) ] = 1
            
        return one_hot_arr
    
    def rbf( self, x, centroids, std ):
        
        dist = distance( x, centroids )
        rbf = numpy.exp( -dist / std**2 )
        
        return rbf
    
    def RBF( self, x ):
    
        self.centroids, std = kmean( x, self.k, self.max_iters )
    
        RBF = []
        for i in range( len( x ) ):
            
            RBF.append( [ self.rbf( x[ i ], c, s ) for ( c, s ) in zip( self.centroids, std ) ] )

            #             for j in range( len( centroids ) ):
            
#                 RBF.append( [ self.rbf( self.input[ i ], centroids[ j ], std[ j ] ) ] )
                
        return numpy.array( RBF )
    
    def train( self ):
        
        RBF = self.RBF( self.input )
        self.weights = numpy.linalg.pinv( RBF.T @ RBF ) @ RBF.T @ self.one_hot_encode( self.y, self.num_classes )
        
    def test( self, test_x, test_y ):
        
        RBF = self.RBF( test_x )
        self.predict = RBF @ self.weights
        self.pred = numpy.array( [ numpy.argmax( x ) for x in self.predict ] )
        self.diff = self.pred - test_y

        print( 'Accuracy: ', len( numpy.where( self.diff == 0 )[ 0 ] ) / len( self.diff ) )
        

In [291]:
def load_data(path):
    with numpy.load(path) as f:
        x_train, y_train = f['x_train'], f['y_train']
        x_test, y_test = f['x_test'], f['y_test']
        return (x_train, y_train), (x_test, y_test)

( x_train, y_train ), ( x_test, y_test ) = load_data( 'mnist.npz' )

train_y = y_train[ :5000 ]
train_x = x_train[ :5000 ]

test_y = y_test[ 0:500 ]
test_x = x_test[ 0:500 ]

In [292]:
num_classes = 10
k = 500
max_iters = 1000


In [293]:
brain = NeuralNet( train_x, train_y, num_classes, k, max_iters )
brain.train()

In [294]:
brain.test( test_x, test_y )


Accuracy:  0.086


In [295]:
test_y

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9,
       8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7,
       9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2,
       6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4,

In [296]:
brain.pred

array([6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6,
       6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 3, 6, 6, 6, 6, 6,
       0, 6, 0, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,

# Questions:

## WHY IS EVERYHTING 6????